# Move and trim files recorded on Feb 14th, trial run

Check and clear 

Plot and review measurements from equivital sensors as output by the Equivital Manager app.
(Every program produces different formated files and it's a pain.)

In [1]:
import sys
import os
import shutil
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import heartpy as hp

from scipy.signal import butter,filtfilt
from scipy import interpolate
from scipy.interpolate import interp1d

## Definitions for data extraction

In [2]:
%load_ext autoreload
%autoreload 1
%aimport qex

In [3]:
%reload_ext autoreload

In [4]:
seperator = str('\\')
seperator

'\\'

# Scan for DATA files

All recordinds exported by Qiosk produce four CSV files and one SEM file. One of the CSVs is a DATA file that contains metadata and some essential signal quality checks on 15 s intervals.

We can ID instances of signal recordings from statistics taken out of the DATA file, so that gets extracted first

In [5]:
# location of the files when exported by QIOSK
path = "C:\\Users\\Public\\Documents\\Equivital\\Equivital Manager Wizard\\"#Extracted SEM Data\\" #"../Equivital/Equivital Manager Wizard/Extracted SEM Data/"
#os.listdir(path)

df_datafiles = qex.qiosk_recordings(path,'Present',seperator)
df_datafiles

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,BR605,5022202,230214,0,DATA-BR605-5022202-23021400.CSV,csv,26127,2023-02-14 12:17:29+00:00,2023-02-14 13:36:59+00:00,...,[BR605],5022202.0,BR605,NaN,46.0,97.0,0.0,1.0,Stationary,Side
1,DATA,BR606,5022203,230214,101,DATA-BR606-5022203-23021400_0.CSV,csv,25515,2023-02-14 12:19:12+00:00,2023-02-14 13:37:12+00:00,...,[BR606],5022203.0,BR606,NaN,20.0,98.0,0.0,1.0,Stationary,Prone
2,DATA,VN208,5022076,230214,101,DATA-VN208-5022076-23021400_0.CSV,csv,28947,2023-02-14 12:20:05+00:00,2023-02-14 13:48:35+00:00,...,[VN208],5022076.0,VN208,NaN,12.0,38.0,0.0,1.0,Stationary,Prone
3,DATA,BR607,5022204,230214,0,DATA-BR607-5022204-23021400.CSV,csv,24318,2023-02-14 12:24:26+00:00,2023-02-14 13:37:11+00:00,...,[BR607],5022204.0,BR607,NaN,106.0,38.0,0.0,1.0,Stationary,Unknown
4,DATA,VN210,5022098,230214,101,DATA-VN210-5022098-23021400_0.CSV,csv,25254,2023-02-14 12:31:54+00:00,2023-02-14 13:48:39+00:00,...,[VN210],5022098.0,VN210,NaN,12.0,99.0,0.0,1.0,Stationary,Prone
5,DATA,VA308,5022131,230214,101,DATA-VA308-5022131-23021400_0.CSV,csv,21652,2023-02-14 12:32:56+00:00,2023-02-14 13:38:56+00:00,...,[VA308],5022131.0,VA308,NaN,10.0,67.0,0.0,1.0,Stationary,Prone
6,DATA,LS408,5022151,230214,101,DATA-LS408-5022151-23021400_0.CSV,csv,19693,2023-02-14 12:34:04+00:00,2023-02-14 13:34:04+00:00,...,[LS408],5022151.0,LS408,NaN,122.0,63.0,0.0,1.0,Stationary,Side
7,DATA,VN106,5022002,230214,0,DATA-VN106-5022002-23021400.CSV,csv,19665,2023-02-14 12:46:05+00:00,2023-02-14 13:45:50+00:00,...,[VN106],5022002.0,VN106,NaN,6.0,73.0,0.0,1.0,Stationary,Prone
8,DATA,WW506,5022166,230214,101,DATA-WW506-5022166-23021400_0.CSV,csv,13480,2023-02-14 12:52:52+00:00,2023-02-14 13:33:37+00:00,...,[WW506],5022166.0,WW506,NaN,32.0,94.0,0.0,1.0,Stationary,Prone
9,DATA,VN107,5022006,230214,102,DATA-VN107-5022006-23021400_1.CSV,csv,10118,2023-02-14 13:15:20+00:00,2023-02-14 13:45:50+00:00,...,[VN107],5022006.0,VN107,NaN,10.0,98.0,0.0,1.0,Stationary,Prone


In [6]:
print('Number of DATA files: '+ str(len(df_datafiles)))
# key categories to structure of Qiosk output DATA files
print(df_datafiles['ID'].unique())
print(df_datafiles['DevName'].unique())
print(df_datafiles['Date'].unique())

Number of DATA files: 47
[5022202 5022203 5022076 5022204 5022098 5022131 5022151 5022002 5022166
 5022006 5022126 5022149 5022009 5023926 5022244 5022163 5022173 5022155
 5022164 5022139 5022240 3420426 5022241 5022077 3420427 5022062 5022057
 5022141 5022127 5022184 3420415 3420456 3420755 5022058 5022050 5022143
 5022051 5022060 5022073]
['BR605' 'BR606' 'VN208' 'BR607' 'VN210' 'VA308' 'LS408' 'VN106' 'WW506'
 'VN107' 'VA306' 'LS407' 'VN108' 'LS406' 'PC704' 'WW504' 'WW507' 'WW501'
 'WW505' 'LS402' 'PC702' 'VN102' 'PC703' 'VN209' 'VN103' 'VN206' 'VN203'
 'LS403' 'VA307' 'BR602' 'VN101' 'VN104' 'VN105' 'VN204' 'VN201' 'LS405'
 'VN202' 'VN205' 'VN207']
[230214 230216]


## Define set of recordings to move

We sort files by day and time recording

In [12]:
# DATA files from recordings on a particular day, YYMMDD
recDate = 230214
s_files=df_datafiles.loc[df_datafiles['Date'] == recDate,:]


Additionally select according to start and stop recording times.

In [13]:
# recordings started before a certain time 
t = pd.to_datetime('2023-02-14 13:59:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecStart']< t,:] 

In [14]:
# recordings started before a certain time 
t = pd.to_datetime('2023-02-14 09:00:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecStart']> t,:] 

In [15]:
# recordings ended after a certain time
t = pd.to_datetime('2023-02-14 14:01:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecEnd']> t,:] 

In [16]:
# show subset of files
s_files

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,BR605,5022202,230214,0,DATA-BR605-5022202-23021400.CSV,csv,26127,2023-02-14 12:17:29+00:00,2023-02-14 13:36:59+00:00,...,[BR605],5022202.0,BR605,NaN,46.0,97.0,0.0,1.0,Stationary,Side
1,DATA,BR606,5022203,230214,101,DATA-BR606-5022203-23021400_0.CSV,csv,25515,2023-02-14 12:19:12+00:00,2023-02-14 13:37:12+00:00,...,[BR606],5022203.0,BR606,NaN,20.0,98.0,0.0,1.0,Stationary,Prone
2,DATA,VN208,5022076,230214,101,DATA-VN208-5022076-23021400_0.CSV,csv,28947,2023-02-14 12:20:05+00:00,2023-02-14 13:48:35+00:00,...,[VN208],5022076.0,VN208,NaN,12.0,38.0,0.0,1.0,Stationary,Prone
3,DATA,BR607,5022204,230214,0,DATA-BR607-5022204-23021400.CSV,csv,24318,2023-02-14 12:24:26+00:00,2023-02-14 13:37:11+00:00,...,[BR607],5022204.0,BR607,NaN,106.0,38.0,0.0,1.0,Stationary,Unknown
4,DATA,VN210,5022098,230214,101,DATA-VN210-5022098-23021400_0.CSV,csv,25254,2023-02-14 12:31:54+00:00,2023-02-14 13:48:39+00:00,...,[VN210],5022098.0,VN210,NaN,12.0,99.0,0.0,1.0,Stationary,Prone
5,DATA,VA308,5022131,230214,101,DATA-VA308-5022131-23021400_0.CSV,csv,21652,2023-02-14 12:32:56+00:00,2023-02-14 13:38:56+00:00,...,[VA308],5022131.0,VA308,NaN,10.0,67.0,0.0,1.0,Stationary,Prone
6,DATA,LS408,5022151,230214,101,DATA-LS408-5022151-23021400_0.CSV,csv,19693,2023-02-14 12:34:04+00:00,2023-02-14 13:34:04+00:00,...,[LS408],5022151.0,LS408,NaN,122.0,63.0,0.0,1.0,Stationary,Side
7,DATA,VN106,5022002,230214,0,DATA-VN106-5022002-23021400.CSV,csv,19665,2023-02-14 12:46:05+00:00,2023-02-14 13:45:50+00:00,...,[VN106],5022002.0,VN106,NaN,6.0,73.0,0.0,1.0,Stationary,Prone
8,DATA,WW506,5022166,230214,101,DATA-WW506-5022166-23021400_0.CSV,csv,13480,2023-02-14 12:52:52+00:00,2023-02-14 13:33:37+00:00,...,[WW506],5022166.0,WW506,NaN,32.0,94.0,0.0,1.0,Stationary,Prone


In [17]:
s_files.sum()

C:\Users\fourMs lab\AppData\Local\Temp\ipykernel_9228\210589667.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  s_files.sum()


Signal                            DATADATADATADATADATADATADATADATADATA
DevName                  BR605BR606VN208BR607VN210VA308LS408VN106WW506
ID                                                            45199233
Date                                                           2071926
Session                                                            606
FileName             DATA-BR605-5022202-23021400.CSVDATA-BR606-5022...
FileType                                   csvcsvcsvcsvcsvcsvcsvcsvcsv
FileSize                                                        204651
Duration                                                       37320.0
BatteryStart                                                     37197
BatteryEnd                                                       36933
BatteryChange(mV)                                                 -264
FullLoc              C:\Users\Public\Documents\Equivital\Equivital ...
SubjectNames           [BR605BR606VN208BR607VN210VA308LS408VN106WW506]
SENSOR

In [18]:
s_files.max()

Signal                                                            DATA
DevName                                                          WW506
ID                                                             5022204
Date                                                            230214
Session                                                            101
FileName                             DATA-WW506-5022166-23021400_0.CSV
FileType                                                           csv
FileSize                                                         28947
RecStart                                     2023-02-14 12:52:52+00:00
RecEnd                                       2023-02-14 13:48:39+00:00
Duration                                                        5310.0
BatteryStart                                                      4158
BatteryEnd                                                        4135
BatteryChange(mV)                                                  -23
FullLo

In [19]:
s_files.min()

Signal                                                            DATA
DevName                                                          BR605
ID                                                             5022002
Date                                                            230214
Session                                                              0
FileName                               DATA-BR605-5022202-23021400.CSV
FileType                                                           csv
FileSize                                                         13480
RecStart                                     2023-02-14 12:17:29+00:00
RecEnd                                       2023-02-14 13:33:37+00:00
Duration                                                        2445.0
BatteryStart                                                      4106
BatteryEnd                                                        4070
BatteryChange(mV)                                                  -41
FullLo

These recordings can be checked in View_Equivital_Qiosk_Export.

Now we move them

# Move selected files to suitable project folder

In [20]:
projectsFolder =  'D:\\' # where qiosk files are moved to after export
print(projectsFolder) # where the qiosk files are initially saved by the Equivital Qiosk program

D:\


In [21]:
# set project folder name
projectName = '230214_Equivital'
projectPath = projectsFolder + projectName + '\\'

In [22]:
discardPath = 'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\ToBeDeleted\\'

### Create folders for project to move files to
Check target location for data files to keep and to remove

In [23]:
# check if the folder exists for project and set up locations
if not os.path.isdir(projectsFolder + projectName):
    os.mkdir(projectsFolder + projectName)
    
if not os.path.isdir(projectsFolder + projectName + '\\' + 'SEM'):
    os.mkdir(projectsFolder + projectName + '\\' + 'SEM')
if not os.path.isdir(projectsFolder + projectName + '\\' + 'CSV'):
    os.mkdir(projectsFolder + projectName + '\\' + 'CSV')

# Set Interval to keep in CSV

We will cut the the CSV files to save on space and presever privacy. 
In this case, we are only taking the 

In [24]:
t1 = pd.to_datetime('2023-02-14 09:55:00.00+0100')# V['DateTime'].iloc[int(len(V)/2)]
t2 = pd.to_datetime('2023-02-14 14:00:00.00+0100')
#t2 = t1+pd.to_timedelta(240,'s')
[t1,t2]

[Timestamp('2023-02-14 09:55:00+0100', tz='pytz.FixedOffset(60)'),
 Timestamp('2023-02-14 14:00:00+0100', tz='pytz.FixedOffset(60)')]

In [25]:
(t2-t1).total_seconds()

14700.0

In [54]:
# test cutting
tic = time.time()

dataLoc = s_files.iloc[0,14]
matched = qex.matched_files(dataLoc,path,seperator) # outputs locations of csv 
for fi in matched:
    if not fi.lower().endswith('.sem'):
        fp = fi.split(seperator)
        fn = fp[-1].split('.')
        fn_new = fn[0]+'-trimmed.csv'
        print(fn_new)
        V = qex.cut_by_time(fi,t1,t2)
        print(V.columns)
        V.to_csv(projectPath + 'CSV\\' + fn_new)
        print(time.time()-tic)
    if eq_file_loc.lower().endswith('sem'):
        if not os.path.isdir(projectsFolder + projectName + '\\SEM\\' + devName):
            os.mkdir(projectsFolder + projectName  + '\\SEM\\' + devName)
        out_f = projectPath + 'SEM\\' + devName  + '\\' + fileName
        os.rename(eq_file_loc,out_f)
        print(time.time()-tic)

DATA-VN201-5022050-23021400-trimmed.csv
Index(['SENSOR ID', 'SUBJECT ID', 'SUBJECT AGE', 'HR(BPM)', 'HRC(%)',
       'BELT OFF', 'LEAD OFF', 'MOTION', 'BODY POSITION', 'BATTERY(mV)',
       'TEMPERATURE', 'BUTTON PRESS'],
      dtype='object')
0.4191780090332031
ECG-VN201-5022050-23021400-trimmed.csv
Index(['Lead 1', 'Lead 2'], dtype='object')
47.62759590148926
FASTACC-VN201-5022050-23021400-trimmed.csv
Index(['Vert Accelerometer', 'Lat Accelerometer', 'Long Accelerometer'], dtype='object')
90.35752868652344
RESPACC-VN201-5022050-23021400-trimmed.csv
Index(['Breathing'], dtype='object')
94.61357927322388
RR-VN201-5022050-23021400-trimmed.csv
Index(['Interbeat Interval (MS)'], dtype='object')
94.8211452960968


# Full Cut Save Move Discard

In [80]:
fn_new

'RR-VN105-3420755-23021400_0-cut.csv'

In [77]:
tac = time.time()

for i,row in s_files.iterrows():
    tic = time.time()
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,path,seperator)  # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                fn = fileName.split('.')
                fn_new = fn[0]+'-cut.csv'
                out_f = projectPath + 'CSV\\' + fn_new
                print(fn_new)
                V = qex.cut_by_time(fi,t1,t2)
                V.to_csv(out_f) # write the trimmed csv to the external harddisk
                
                dout_f = discardPath + 'CSV\\' + fileName
                shutil.move(fi,dout_f) # move the exported csv files to the discard folder
                print(time.time()-tic)
        if fileName.lower().endswith('sem'):
            devName = row['DevName']
            if not os.path.isdir(projectsFolder + projectName + '\\SEM\\' + devName):
                os.mkdir(projectsFolder + projectName  + '\\SEM\\' + devName)
            out_f = projectPath + 'SEM\\' + devName  + '\\' + fileName
            shutil.move(fi,out_f)
            print(time.time()-tic)
            
    print([time.time()-tic, time.time()-tac])
            

DATA-LS401-5022138-23021400-cut.csv
0.04306817054748535
ECG-LS401-5022138-23021400-cut.csv
24.973804712295532
FASTACC-LS401-5022138-23021400-cut.csv
47.11313819885254
RESPACC-LS401-5022138-23021400-cut.csv
49.26677179336548
RR-LS401-5022138-23021400-cut.csv
49.38849496841431
50.68850517272949
[50.68850517272949, 50.68950629234314]
DATA-LS402-5022139-23021400-cut.csv
0.03800034523010254
ECG-LS402-5022139-23021400-cut.csv
25.28141713142395
FASTACC-LS402-5022139-23021400-cut.csv
47.61280012130737
RESPACC-LS402-5022139-23021400-cut.csv
49.71704053878784
RR-LS402-5022139-23021400-cut.csv
49.86404085159302
51.01604127883911
[51.01604127883911, 101.70554757118225]
DATA-VA301-5022100-23021400_0-cut.csv
0.03699994087219238
ECG-VA301-5022100-23021400_0-cut.csv
24.76331090927124
FASTACC-VA301-5022100-23021400_0-cut.csv
46.8141770362854
RESPACC-VA301-5022100-23021400_0-cut.csv
48.90080761909485
RR-VA301-5022100-23021400_0-cut.csv
49.01880955696106
50.19780874252319
[50.19780874252319, 151.90335631

44.783915281295776
RESPACC-LS405-5022143-23021400-cut.csv
46.8218936920166
RR-LS405-5022143-23021400-cut.csv
46.93872022628784
48.06985926628113
[48.06985926628113, 1252.3221442699432]
DATA-WW503-5022161-23021400-cut.csv
0.03500032424926758
ECG-WW503-5022161-23021400-cut.csv
23.742379903793335
FASTACC-WW503-5022161-23021400-cut.csv
44.70373463630676
RESPACC-WW503-5022161-23021400-cut.csv
46.70140290260315
RR-WW503-5022161-23021400-cut.csv
46.86739921569824
48.06947112083435
[48.06947112083435, 1300.3916153907776]
DATA-VN204-5022058-23021400-cut.csv
0.03500032424926758
ECG-VN204-5022058-23021400-cut.csv
23.738090991973877
FASTACC-VN204-5022058-23021400-cut.csv
44.73535180091858
RESPACC-VN204-5022058-23021400-cut.csv
46.751171588897705
RR-VN204-5022058-23021400-cut.csv
46.86817121505737
47.94817233085632
[47.94817233085632, 1348.339787721634]
DATA-VN209-5022077-23021400-cut.csv
0.03499960899353027
ECG-VN209-5022077-23021400-cut.csv
23.834912300109863
FASTACC-VN209-5022077-23021400-cut.cs

TypeError: qiosk_recordings() missing 1 required positional argument: 'sep'

In [82]:
dfiles = qex.qiosk_recordings(projectPath,projectName,seperator)
print(projectPath)
print(projectName)
dfiles

D:\230214_Equivital\
230214_Equivital


,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,VA304,5022110,230214,0,DATA-VA304-5022110-23021400-cut.csv,csv,80491,2023-02-14 08:55:02+00:00,2023-02-14 12:59:47+00:00,...,[VA304],5022110.0,VA304,NaN,63,98.0,0.0,0.0,Stationary,Upright
1,DATA,BR604,5022198,230214,101,DATA-BR604-5022198-23021400_0-cut.csv,csv,80215,2023-02-14 08:55:04+00:00,2023-02-14 12:59:49+00:00,...,[BR604],5022198.0,BR604,NaN,65,98.0,0.0,0.0,Stationary,Upright
2,DATA,VN205,5022060,230214,0,DATA-VN205-5022060-23021400-cut.csv,csv,80246,2023-02-14 08:55:04+00:00,2023-02-14 12:59:49+00:00,...,[VN205],5022060.0,VN205,NaN,63,99.0,0.0,0.0,Stationary,Upright
3,DATA,PC702,5022240,230214,0,DATA-PC702-5022240-23021400-cut.csv,csv,80308,2023-02-14 08:55:04+00:00,2023-02-14 12:59:49+00:00,...,[PC702],5022240.0,PC702,NaN,70,98.0,0.0,0.0,Stationary,Upright
4,DATA,VN202,5022051,230214,0,DATA-VN202-5022051-23021400-cut.csv,csv,80336,2023-02-14 08:55:04+00:00,2023-02-14 12:59:51+00:00,...,[VN202],5022051.0,VN202,NaN,84,99.0,0.0,0.0,Stationary,Upright
5,DATA,PC703,5022241,230214,0,DATA-PC703-5022241-23021400-cut.csv,csv,80996,2023-02-14 08:55:05+00:00,2023-02-14 12:59:50+00:00,...,[PC703],5022241.0,PC703,NaN,74,100.0,0.0,0.0,Stationary,Upright
6,DATA,LS401,5022138,230214,0,DATA-LS401-5022138-23021400-cut.csv,csv,80302,2023-02-14 08:55:05+00:00,2023-02-14 12:59:50+00:00,...,[LS401],5022138.0,LS401,NaN,70,99.0,0.0,0.0,Stationary,Upright
7,DATA,PC701,5022238,230214,0,DATA-PC701-5022238-23021400-cut.csv,csv,80391,2023-02-14 08:55:05+00:00,2023-02-14 12:59:50+00:00,...,[PC701],5022238.0,PC701,NaN,80,98.0,0.0,0.0,Stationary,Upright
8,DATA,WW502,5022156,230214,0,DATA-WW502-5022156-23021400-cut.csv,csv,80378,2023-02-14 08:55:06+00:00,2023-02-14 12:59:51+00:00,...,[WW502],5022156.0,WW502,NaN,70,98.0,0.0,0.0,Stationary,Upright
9,DATA,LS403,5022141,230214,0,DATA-LS403-5022141-23021400-cut.csv,csv,80401,2023-02-14 08:55:07+00:00,2023-02-14 12:59:52+00:00,...,[LS403],5022141.0,LS403,NaN,88,98.0,0.0,0.0,Stationary,Upright


## Troubleshooting

In [69]:
shutil.move(fi,out_f)

'D:\\230214_Equivital\\SEM\\VN101\\23021400.SEM'

In [11]:
dfiles = qex.qiosk_recordings(discardPath, 'ToBeDiscarded',seperator)
print(len(dfiles))

585
